Code to preprocess videos into their flow_x and flow_y images and return normal frames.

In [ ]:
import os,sys
import numpy as np
import cv2
import matplotlib
from PIL import Image
import multiprocessing as mp
import threading

data_root = '/content/drive/MyDrive/flow_test2'
bound = 15
print(f'Number of processors: {mp.cpu_count()}')

def ToImg(raw_flow, bound):
    '''
    rescales image pixels to the range 0-255 to by boundry scaling.
    '''
    flow = raw_flow
    flow[flow>bound] = bound
    flow[flow<-bound] = -bound
    flow -= -bound
    flow *= (255/float(2*bound))
    return flow

def save_flows(flows,image,img_save_loc,save_dir,num,bound):
    '''
    Saves x and y flows as well as default frame images to data_dir.
    :return: return 0
    '''
    #rescale to 0~255 with the bound setting
    flow_x = ToImg(flows[...,0],bound)
    flow_y = ToImg(flows[...,1],bound)
    if not os.path.exists(os.path.join(img_save_loc, save_dir)):
        os.makedirs(os.path.join(img_save_loc, save_dir))

    #save images
    save_img = os.path.join(img_save_loc,save_dir, f'img_{num:05d}.jpg')
    matplotlib.image.imsave(save_img, image)

    #save flows
    save_x = os.path.join(img_save_loc, save_dir, f'flow_x_{num:05d}.jpg')
    save_y = os.path.join(img_save_loc, save_dir, f'flow_y_{num:05d}.jpg')
    flow_x_img = Image.fromarray(flow_x)
    flow_y_img = Image.fromarray(flow_y)
    matplotlib.image.imsave(save_x, flow_x_img)
    matplotlib.image.imsave(save_y, flow_y_img)
    return 0


def val_optical_prep(file_loc, label, size):
    '''
    takes a video file location, video label, and desired frame size.

    file_loc (str): location of video to preprocess

    label (str): dtring of classification label

    size (tuple): tuple of with and height dims of image resize

    returns: flow_x/y and normal images to predefined data_root folder
    '''

    dir_name = file_loc.split('/')[5].split('_')[0]
    img_save_loc = os.path.join(data_root, label)
    if not os.path.exists(os.path.join(data_root, label)):
        os.makedirs(os.path.join(data_root,label))
    
    resize = size

    cap = cv2.VideoCapture(str(file_loc))
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_num = 0
    ret = True
      
    frames=[]

    image,prev_image,gray,prev_gray=None,None,None,None
    num0=0

    #get all video frames
    while ret == True:

        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame,resize)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame/255.0
            frame = np.array(frame, dtype=np.float32)
            frames.append(frame)

    for frame in frames:

        #for first frame where there is no previous frame
        if frame_num==0:
            image=np.zeros_like(frame)
            gray=np.zeros_like(frame)
            prev_gray=np.zeros_like(frame)                
            prev_image = frame
            prev_gray = cv2.cvtColor(prev_image,cv2.COLOR_RGB2GRAY)
            frame_num += 1
            continue

        image = frame1
        gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

        frame_0 = prev_gray
        frame_1 = gray

        # the flow algorithm outlined in https://github.com/deepmind/kinetics-i3d
        dtvl1=cv2.optflow.createOptFlow_DualTVL1()
        flowDTVL1=dtvl1.calc(frame_0, frame_1, None)

        #saves both flow image frames and standard frames
        save_flows(
            flowDTVL1, 
            image, 
            img_save_loc, 
            dir_name, 
            frame_num, 
            bound
        )

        prev_gray = gray
        prev_image = image
        frame_num += 1

    cap.release()

    cv2.destroyAllWindows()
    
    # print that video is completed
    return print('DONE!')

Number of processors:  4


In [ ]:
import pandas as pd
import glob
import re
import math

In [ ]:
files = glob.glob('/content/drive/MyDrive/training_arr/*.avi') 
files_val = glob.glob('/content/drive/MyDrive/validation_arr/*.avi')

training_labels = []
training_files = []
val_labels = []
val_files = []

unseen = ['recieve']

In [ ]:
for file in files:
    label = re.findall('[A-Za-z]+[0-9]',str(file))[0][:-1]
    training_labels.append(label)
    training_files.append(str(file))
    
for file in files_val:
    label = re.findall('[A-Za-z]+[0-9]',str(file))[0][:-1]
    val_labels.append(label)
    val_files.append(str(file))

In [ ]:
training_data = pd.DataFrame({'filename':training_files,'training_labels':training_labels})
val_data = pd.DataFrame({'filename':val_files,'val_labels':val_labels})
#removing words that dont appear in training set
val_data = val_data[val_data['val_labels'] != 'receive']

In [ ]:
def quick_make(category, folder, size, start, end):
    '''
    set up preprocessing function for multithreading
    '''
    for i in range(start,end):
        val_optical_prep(category.iloc[i], folder.iloc[i], (224,224))
        print(i)

def split_processing(category, folder, size, num_splits=mp.cpu_count()):
    '''
    function to multithread flow preprocessing

    category (list): list of strings pointing to file locations
    folder (list): list of strings of corresponding labels
    size (tuple): tuple of image resize params (width, height)
    num_splits (int): processors to multithread with
    '''
    split_size = len(category) // num_splits
    threads = []
    for i in range(num_splits):
        # determine the indices of the list this thread will handle
        start = i * split_size
        # special case on the last chunk to account for uneven splits
        end = None if i+1 == num_splits else (i+1) * split_size
        # create the thread
        threads.append(
            threading.Thread(target=quick_make, args=(category, folder, size, start, end)))
        threads[-1].start() # start the thread we just created

    # wait for all threads to finish
    for t in threads:
        t.join()


In [ ]:
split_processing(training_data.filename, training_data.training_labels, (224,224), num_splits=mp.cpu_count())

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-154a02908673>", line 2, in quick_make
    for i in range(start,end):
TypeError: 'NoneType' object cannot be interpreted as an integer

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-154a02908673>", line 3, in quick_make
    val_optical_prep(category.iloc[i], folder.iloc[i], (224,224))
  File "<ipython-input-1-2a723598e5c9>", line 61, in val_optical_prep
    os.makedirs(os.path.join(data_root,label))
  File "/usr/lib/python3.7/os.py", line 223, in makedirs
    mkdir(name, mode)

DONE!
0
